# 04 Optimize

In [2]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

In [3]:
df = pd.read_csv('../data/opt_input.csv')
df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,Strong_Win,Orig_Seed_Strong,Orig_Seed_Weak,Matchup,Round_v2,comp,Key,Win_Prob
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,1,1,16,15,1,1,115,0.992647
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,0,1,16,15,1,-1,115,0.007353
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,1,1,8,7,2,1,207,0.811594
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,0,1,8,7,2,-1,207,0.188406
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,1,16,8,8,2,-1,208,0.254310


In [4]:
for c in set(df['Slot']):
    df.loc[df['Slot']==c, 'Round_Game_'+c] = 1
    df['Round_Game_'+c].fillna(0, inplace=True)

In [5]:
teams = set(list(df['TeamID_Strong']) + list(df['TeamID_Weak']))

In [6]:
def set_value(rnd, win_tm, w_tm, s_tm, r, t):
#     if rnd==6:
#         if win_tm!=t:
#             if (w_tm==t or s_tm==t):
#                 return 6
    if rnd==r:
        if (w_tm==t or s_tm==t):
#         if win_tm==t:
            return r
    if rnd<r:
        if win_tm==t:
            return -1
    return 0
set_value_v = np.vectorize(set_value, excluded=['r', 't'])

    

for r in set(df['Round']):
    for t in teams:
#         df[str(t)+'_'+str(r)] = [0 if t!=tm else(a if a==r else (-1 if a<r else 0)) for a,tm in zip(df['Round'], df['TeamID_Winner'])]
        df[str(t)+'_'+str(r)] = set_value_v(df['Round'], df['TeamID_Winner'], df['TeamID_Weak'], df['TeamID_Strong'], r, t)

In [7]:
# df = df.loc[(df['Slot']=='R1W1')|(df['Slot']=='R1W8')|(df['Slot']=='R2W1') ]

# df = df.loc[df['Round']<=5].reset_index(drop=True)

In [8]:
solver = pywraplp.Solver('SolveIntegerProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
x_list = {}
for i in range(len(df)):
    x_list[i] = solver.IntVar(0.0, 1.0, 'x'+str(i))   

In [9]:
# sets constraints

# add game constraints
for rg in set(df['Slot']):
    constraint = solver.Constraint(1, 1)
    for i in range(len(df)):
        constraint.SetCoefficient(x_list[i], int(df.loc[i, 'Round_Game_'+rg]))

# adds constraint for each team/round
for r in set(df['Round']):
    for t in teams:
        col = str(t)+'_'+str(r)
        constraint = solver.Constraint(-50, 1)
        for i in range(len(df)):
           constraint.SetCoefficient(x_list[i], int(df.loc[i, col]))

In [10]:
 # sets objective
opt_vars = 'Win_Prob'    
    
obj = solver.Objective()
for i in range(len(df)):
    obj.SetCoefficient(x_list[i], int(df.loc[i, opt_vars]))
obj.SetMaximization()

In [11]:
result_status = solver.Solve()
result_status

0

In [12]:
df

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,1139_6,1395_6,1138_6,1397_6,1400_6,1401_6,1274_6,1403_6,1276_6,1277_6
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,0,0,0,0,0,0,0,0,0,0
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,...,0,0,0,0,0,0,0,0,0,0
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,0,0,0,0,0,0,0,0,0,0
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,...,0,0,0,0,0,0,0,0,0,0
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,...,0,0,0,0,0,0,0,0,0,0
5,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1439,...,0,0,0,0,0,0,0,0,0,0
6,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1104,Alabama,1437,...,0,0,0,0,0,0,0,0,0,0
7,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1104,Alabama,1104,...,0,0,0,0,0,0,0,0,0,0
8,2018,R1W8,W08,W09,1,1439,Virginia Tech,1104,Alabama,1439,...,0,0,0,0,0,0,0,0,0,0
9,2018,R1W8,W08,W09,1,1439,Virginia Tech,1104,Alabama,1104,...,0,0,0,0,0,0,0,0,0,0


In [13]:
opt_results = df.copy()

for i in range(len(df)):
    opt_results.loc[i, 'Play'] = x_list[i].solution_value()
    
opt_results = opt_results.loc[opt_results['Play']==1][['Season', 'Round', 'Slot', 'TeamName_Strong', 'TeamName_Weak', 'Orig_Seed_Strong', 'Orig_Seed_Weak', 'Win_Prob', 'Strong_Win']]


In [14]:
opt_results

,Season,Round,Slot,TeamName_Strong,TeamName_Weak,Orig_Seed_Strong,Orig_Seed_Weak,Win_Prob,Strong_Win
0,2018,1,R1W1,Villanova,Radford,1,16,0.992647,1
2,2018,2,R2W1,Villanova,Virginia Tech,1,8,0.811594,1
8,2018,1,R1W8,Virginia Tech,Alabama,8,9,0.500000,1
23,2018,1,R1W4,Wichita St,Marshall,4,13,0.205882,0
42,2018,3,R3W1,Villanova,Murray St,1,12,1.000000,1
47,2018,1,R1W5,West Virginia,Murray St,5,12,0.345588,0
53,2018,2,R2W4,Marshall,Murray St,13,12,0.578313,0
73,2018,4,R4W1,Villanova,CS Fullerton,1,15,0.500000,0
75,2018,1,R1W2,Purdue,CS Fullerton,2,15,0.058824,0
102,2018,2,R2W2,CS Fullerton,Arkansas,15,7,0.254310,1


In [16]:
opt_results.to_csv('../data/opt_results.csv', index=False)

In [15]:
df.to_csv('../data/opt_input_v2.csv', index=False)

In [16]:
# not quite sure why it isn't working.  